## 第二節　ダクト式セントラル空調機

1．適用範囲～4．記号及び単位は省略

### 5．最大暖房出力

$$\large
Q_{max,H,d,t} = q_{rtd,H} \times C_{df,H,d,t} \times 3600 \times 10^{-6}
$$

$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの最大暖房出力(MJ/h)  
$q_{rtd,H}$：定格暖房能力(W)   
$C_{df,H,d,t}$：日付$d$の時刻$t$におけるデフロストに関する暖房出力補正係数  

日付$d$の時刻$t$におけるデフロストに関する暖房出力補正係数$C_{df,H,d,t}$は、外気温度が5℃未満かつ相対湿度が80%以上の場合にデフロストが入ると仮定し、その場合の値を0.77とし、それ以外の条件においては1.0とする。

### 6．暖房エネルギー消費量

#### 6.1　消費電力量

$Qr'_{T,H,d,t} \geq 0.3$ の場合

$$
\large
E_{E,H,d,t} = ( a_1 \times ( b_1 \times \theta_{ex,d,t} + b_2 \times ln(Qr'_{T,H,d,t}) + b_3 )^{a2}  \\
\large
+ c_1 \times \Biggl( \frac{Qr'_{T,H,d,t}}{d_1 \times \theta_{ex,d,t} + d_2 \times ln(Qr'_{T,H,d,t}) + d_3} \Biggr)^{c_2} \Biggr)
\times P_{rtd,H} \times 10^{-3} \times C_{pl,H}
$$

$Qr'_{T,H,d,t} < 0.3$ の場合

$$
\large
E_{E,H,d,t} = ( a_1 \times ( b_1 \times \theta_{ex,d,t} + b_2 \times ln(0.3) + b_3 )^{a_2}  \\
\large
+ c_1 \times \Biggl( \frac{0.3}{d_1 \times \theta_{ex,d,t} + d_2 \times ln(0.3) + d_3} \Biggr)^{c_2} \Biggr)
\times P_{rtd,H} \times \frac{Qr'_{T,H,d,t}}{0.3} \times 10^{-3} \times C_{pl,H}
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h)  
$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)  
$Qr'_{T,H,d,t}$：日付$d$の時刻$t$における補正処理暖房負荷比    
$P_{rtd,H}$：定格暖房消費電力(W)  
$C_{pl,H}$：暖房時におけるダクト等圧力損失による消費電力量補正係数  

<div style="text-align: center;font-weight: bold;">表　係数</div>

|係数|値|係数|値|
|-|-|-|-|
|$a_1$|0.0003124|$c_1$|0.003955|
|$a_2$|1.923|$c_2$|2.001|
|$b_1$|-0.7945|$d_1$| 7.442 $\times 10^{-5}$|
|$b_2$|21.54|$d_2$|0.09156|
|$b_3$|73.02|$d_3$|0.2344|


$$\large
Qr'_{T,H,d,t} = \frac{Q'_{T,H,d,t}}{q_{rtd,H} \times 3600 \times 10^{-6}}
$$

$q_{rtd,H,d,t}$：定格暖房能力(W)  
$Q'_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの補正処理暖房負荷(MJ/h)    

$$\large
Q'_{T,H,d,t} = \frac{Q_{T,H,d,t}}{C_{df,H,d,t}}
$$

$Q_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの処理暖房負荷(MJ/h)  
$C_{df,H,d,t}$：日付$d$の時刻$t$におけるデフロストに関する暖房出力補正係数    

日付$d$の時刻$t$におけるデフロストに関する暖房出力補正係数$C_{df,H,d,t}$は、外気温度が5℃未満かつ相対湿度が80%以上の場合にデフロストが入ると仮定し、その場合の値を0.77とし、それ以外の条件においては1.0とする。

#### 6.2　ガス消費量

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,H,d,t}$は0とする。

#### 6.3　灯油消費量

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,H,d,t}$は0とする。

#### 6.4　その他の燃料による一次エネルギー消費量

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,H,d,t}$は0とする。

### 7．最大冷房出力

#### 7.1　最大冷房出力の計算

$$\large
Q_{max,C,d,t} = q_{rtd,C} \times 3600 \times 10^{-6}
$$

$Q_{max,C,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房出力(MJ/h)  
$q_{rtd,C}$：定格冷房能力(W)  

#### 7.2　最大冷房顕熱出力及び最大冷房潜熱出力の計算

$$\large
Q_{max,CS,d,t} = Q_{max,C,d,t} \times SHF'_{d,t}
$$

$$\large
Q_{max,CL,d,t} = min(Q_{max,C,d,t} \times (1 - SHF'_{d,t}),  L'_{CL,d,t})
$$

$Q_{max,CS,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房顕熱出力(MJ/h)  
$Q_{max,CL,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房潜熱出力(MJ/h)  
$SHF'_{d,t}$：日付$d$の時刻$t$における冷房負荷補正顕熱比  
$L'_{CL,d,t}$：日付$d$の時刻$t$における1時間当たりの補正冷房潜熱負荷(MJ/h)  
$Q_{max,C,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房出力(MJ/h)  

$$\large
SHF'_{d,t} = \frac{L_{CS,d,t}}{L'_{C,d,t}}
$$

$L_{CS,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房顕熱負荷(MJ/h)  
$L'_{C,d,t}$：日付$d$の時刻$t$における1時間当たりの補正冷房負荷(MJ/h)  

日付$d$の時刻$t$における１時間当たりの補正冷房負荷$L'_{C,d,t}$が0の場合、$SHF'_{d,t}$ = 0とする。

$$\large
L'_{c,d,t} = L_{CS,d,t} + L'_{CL,d,t}
$$

$$\large
L'_{CL,d,t} = min(L_{max,CL,d,t},  L_{CL,d,t})
$$

$$\large
L_{max,CL,d,t} = L_{CS,d,t} \times \frac{1 - SHF_{L,min,C}}{SHF_{L,min,C}}
$$

$L_{max,CL,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房潜熱負荷(MJ/h)  
$L_{CL,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房潜熱負荷(MJ/h)  
$SHF_{L,min,C}$：冷房負荷最小顕熱比  

冷房負荷最小顕熱比$SHF_{L,min,C}$は0.4 とする。

### 8．冷房エネルギー消費量

#### 8.1　消費電力量

$Qr_{T,C,d,t} \geq 0.3$ の場合

$$\large
E_{E,C,d,t} = (a_1 \times (b)^{a_2} + c_1 \times (d)^{c_2}) \times P_{rtd,C} \times 10^{-3} \times C_{pl,C}
$$

$$\large
\left\{ \begin{array}{ll}
    b = max(\theta_{ex,d,t} - b_1 \times \theta_{ex,d,t} - b_2 \times ln(X_{ex,d,t}) - b_3 \times ln(Qr_{T,C,d,t}) - b_{4},0)\\ 
    d = exp(d_1 \times \theta_{ex,d,t} + d_2 \times ln(X_{ex,d,t}) + d_3 \times ln(Qr_{T,C,d,t}) + d_4)\\ 
    \end{array}
\right.
$$

$Qr_{T,C,d,t} < 0.3$ の場合

$$\large
E_{E,C,d,t} = (a_1 \times (b)^{a_2} + c_1 \times (d)^{c_2}) \times P_{rtd,C} \times \frac{Qr_{T,C,d,t}}{0.3} \times 10^{-3} \times C_{pl,C}
$$

$$\large
\left\{ \begin{array}{ll}
    b = max(\theta_{ex,d,t} - b_1 \times \theta_{ex,d,t} - b_2 \times ln(X_{ex,d,t}) - b_3 \times ln(0.3) - b_{4},0)\\ 
    d = exp(d_1 \times \theta_{ex,d,t} + d_2 \times ln(X_{ex,d,t}) + d_3 \times ln(0.3) + d_4)\\ 
    \end{array}
\right.
$$

$E_{E,C,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h)  
$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)    
$X_{ex,d,t}$：日付$d$の時刻$t$における外気絶対湿度(kg/kg(DA))    
$Qr_{T,C,d,t}$：日付$d$の時刻$t$における処理冷房負荷比    
$P_{rtd,C}$：定格冷房消費電力(W)    
$C_{pl,C}$：冷房時におけるダクト等圧力損失による消費電力量補正係数  

<div style="text-align: center;font-weight: bold;">表　係数</div>

|係数|値|係数|値|
|-|-|-|-|
|$a_1$|0.001052|$c_1$|0.003939|
|$a_2$|2.051|$c_2$|1.058|
|$b_1$|0.241|$d_1$|0.0593|
|$b_2$|0.04985|$d_2$|0.08193|
|$b_3$|0.2083|$d_3$|1.257|
|$b_4$|−2.391|$d_4$|1.586|


$$\large
Qr_{T,C,d,t} = \frac{Q_{T,C,d,t}}{q_{rtd,C} \times 3600 \times 10^{-6}}
$$

$Q_{T,C,d,t}$：日付$d$の時刻$t$における1時間当たりの処理冷房負荷(MJ/h)  
$q_{rtd,C}$：定格冷房能力(W)

$$\large
Q_{T,C,d,t} = Q_{T,CS,d,t} + Q_{T,CL,d,t}
$$

$Q_{T,CS,d,t}$：日付$d$の時刻$t$における1時間当たりの処理冷房顕熱負荷(MJ/h)  
$Q_{T,CL,d,t}$：日付$d$の時刻$t$における1時間当たりの処理冷房潜熱負荷(MJ/h)  

#### 8.2　ガス消費量

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,C,d,t}$は0とする。

#### 8.3　灯油消費量

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,C,d,t}$は0とする。

#### 8.4　その他の燃料による一次エネルギー消費量

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,C,d,t}$は0とする。

## 付録 A 機器の性能を表す仕様の決定方法

### A.1　定格能力

$$\large
q_{rtd,H} = q_{rq,H} \times A_{A} \times f_{CT} \times f_{CL}
$$

$$\large
q_{rtd,C} = q_{rq,C} \times A_{A} \times f_{CT} \times f_{CL}
$$

$q_{rtd,H}$：定格暖房能力(W)  
$q_{rtd,C}$：定格冷房能力(W)  
$q_{rq,H}$：単位面積当たりの必要暖房能力(W/m<sup>2</sup>)  
$q_{rq,C}$：単位面積当たりの必要冷房能力(W/m<sup>2</sup>)  
$A_{A}$：床面積の合計(m<sup>2</sup>)   
$f_{CT}$：外気温度能力補正係数   
$f_{CL}$：間歇運転能力補正係数  

外気温度能力補正係数$f_{CT}$は、1.05の値とする。間歇運転能力補正係数$f_{CL}$は、1.0とする。

<div style="text-align: center;font-weight: bold;">表　単位面積当たりの必要暖房能力$q_{rq,H}$及び冷房能力$q_{rq,C}$(W/m<sup>2</sup>)</div>

|変数＼地域の区分|1|2|3|4|5|6|7|8|
|-|-|-|-|-|-|-|-|-|
|$q_{rq,H}$|73.91|64.32|62.65|66.99|72.64|61.34|64.55|－|
|$q_{rq,C}$|37.61|36.55|42.34|54.08|61.69|60.79|72.53|61.56|


### A.2　定格消費電力

$$\large
P_{rtd,H} = \frac{q_{rtd,H}}{e_{rtd,H}}
$$

$$\large
P_{rtd,C} = \frac{q_{rtd,C}}{e_{rtd,C}}
$$

$P_{rtd,H}$：定格暖房消費電力(W)  
$P_{rtd,C}$：定格冷房消費電力(W)  
$q_{rtd,H}$：定格暖房能力(W)  
$q_{rtd,C}$：定格冷房能力（(W)  
$e_{rtd,H}$：定格暖房エネルギー消費効率  
$e_{rtd,C}$：：定格冷房エネルギー消費効率  

定格暖房エネルギー消費効率$e_{rtd,H}$は3.76、定格冷房エネルギー消費効率$e_{rtd,C}$は3.17とする。

### A.3　複数のダクト式セントラル空調機が設置される場合の仕様の決定方法

省略

## 付録 B ダクト等圧力損失及び断熱区画外ダクト熱損失による消費電力量補正係数

pythonを確認してから記述するため、現時点では保留